In [ ]:
# conda install pymysql

In [ ]:
import re

In [ ]:
import os

# folder path
dir_path = r'/home/trishalaswain/Downloads/JanSCP18/JanSCP-New18v3Selected/'

# list to store files
res = []

# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)) and not str(path).endswith("HCPCS.tsv") and not str(path).endswith("CPT.tsv") and not str(path).endswith("CM.tsv")  and not str(path).endswith("EANDM.tsv"):

        # print(path)
        
        res.append(dir_path + path)
        # print(path)

In [ ]:
df_final = pd.DataFrame()

In [ ]:
def initialize(index):
    df_final.at[index,'1_doc_only_counts'] = 0
    df_final.at[index,'2_doc_only_counts'] = 0
    df_final.at[index,'3_doc_only_counts'] = 0
    df_final.at[index,'4_doc_only_counts'] = 0
    df_final.at[index,'other_no_of_docs_counts'] = 0

In [ ]:
# df_final['facility_code'] = None
df_final['facility_name'] = None
# df_final['prod_id'] = None
df_final['no_of_encounters'] = None
df_final['total_docs'] = None
df_final['doc_mean'] = 0
df_final['doc_std_dev'] = 0
df_final['25_percentile'] = 0
df_final['50_percentile'] = 0
df_final['75_percentile'] = 0
df_final['min_doc_count'] = 0
df_final['max_doc_count'] = 0

In [ ]:
def fill_df_final(index,facility_name,total_no_of_encounters,total_no_of_docs):
    df_final.at[index,'facility_name'] = facility_name
    # df_final.at[index,'individual_doc_counts'] = docs
    df_final.at[index,'no_of_encounters'] = total_no_of_encounters
    df_final.at[index,'total_docs'] = total_no_of_docs
    initialize(index)

In [ ]:
for file in res:
    
    filename = re.findall("SCP-\d\d.*", file)
    # print(str(filename))
    df = pd.read_table(file, index_col = False)
    for index,row in df.iterrows():
        df1 = df['encDocIds'].dropna
        df1.shape
        index = 0
        for index, r in df1.iterrows():
            
            
        

In [ ]:
df1

In [ ]:
index = 0

for index,row in df.iterrows():
    # filename = str(row['facility_code']) + "_" + str(row['facility_name']) + "_" + str(row['prod_id']) +  ".xlsx"

    sql = """SELECT 
        em.id AS encounter_id,
        am.account_number,
        COUNT(dm.id) AS document_count
    FROM
        encounter_mst em
            INNER JOIN
        account_mst am ON am.id = em.account_id
            INNER JOIN
        document_mst dm ON dm.account_id = am.id
    WHERE
        is_physician_document = 1
            AND am.facility_id = """  + str(row['prod_id']) + """
            AND dm.created_date > '2023-01-01'
    GROUP BY account_number
    ORDER BY document_count DESC;"""
    
    
    sql_query = pd.read_sql(sql,con = connection)
    total_no_of_encounters = sql_query['encounter_id'].count()
    total_no_of_docs = sql_query['document_count'].sum()
    fill_df_final(index, row['facility_code'], row['facility_name'], row['prod_id'], total_no_of_encounters, total_no_of_docs)
    
        
#   Finding count of encounters with only1,only2,only3,only4 & more than4 docs
    for doc_counts in sql_query.document_count:
        if doc_counts == 1:
            df_final.at[index,'1_doc_only_counts'] += 1  
            # print(type(df_final.at[index,'1_doc_only_counts']))
        if doc_counts == 2:
            df_final.at[index,'2_doc_only_counts'] += 1   
        if doc_counts == 3:
            df_final.at[index,'3_doc_only_counts'] += 1   
        if doc_counts == 4:
            df_final.at[index,'4_doc_only_counts'] += 1   
        else:
            df_final.at[index,'other_no_of_docs_counts'] += 1  
            
    sql_query = sql_query.sort_values(by=['document_count'], ascending=True).reset_index()
    
    #   Finding mean of doc_counts
    mean = sql_query['document_count'].mean()
    df_final.at[index,'doc_mean'] = mean

    #   Finding standard deviation of doc_counts
    std_dev = sql_query['document_count'].std()
    df_final.at[index,'doc_std_dev'] = std_dev

    #   Finding 25th percentile
    _25_percentile = sql_query['document_count'].quantile(0.25)
    df_final.at[index,'25_percentile'] = _25_percentile

    #   Finding 50th percentile
    _50_percentile = sql_query['document_count'].quantile(0.50)
    df_final.at[index,'50_percentile'] = _50_percentile

    #   Finding 75th percentile
    _75_percentile = sql_query['document_count'].quantile(0.75)
    df_final.at[index,'75_percentile'] = _75_percentile

    #   Finding minimum doc_counts of all
    minimum = sql_query['document_count'].min()
    df_final.at[index,'min_doc_count'] = minimum

    #   Finding maximum doc_counts of all
    maximum = sql_query['document_count'].max()
    df_final.at[index,'max_doc_count'] = maximum

    columns_titles = ['facility_code','facility_name','prod_id','no_of_encounters','total_docs','1_doc_only_counts','2_doc_only_counts'
                      ,'3_doc_only_counts','4_doc_only_counts','other_no_of_docs_counts','doc_mean','doc_std_dev','25_percentile','50_percentile'
                      ,'75_percentile','min_doc_count','max_doc_count']
      
    df_final = df_final.reindex(columns=columns_titles)

    index = index + 1
df_final.to_excel("/home/trishalaswain/Downloads/Data/document_stats_report.xlsx",index = False)
print("created successfully")

In [ ]:
df_final

In [ ]:
# sql_query['document_count'].describe(percentiles=0.25)

In [ ]:
sql_query['encounter_id'].unique().size


In [ ]:
sql_query.info()